<a href="https://colab.research.google.com/github/WilyLynx/Piekna_corona_python/blob/master/Part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plotly

## Mapy w plotly

### GEOJSON

Na początek potrzebujemy pliku w formacie GEOJSON, który zawiera kontury interesujących nas obszarów. W tym celu wykorzystamy pliki znajdujące się na tym [repozytorium](https://github.com/ppatrzyk/polska-geojson).

In [0]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/ppatrzyk/polska-geojson/master/powiaty/powiaty-min.geojson') as response:
    powiaty = json.load(response)

powiaty["features"][0]

Jak widzimy na dane poszczególnych powiatow składaja się lista wielokąt reprezentujący powiat. Dla nas interesującą właściwością jest pole id wewnątrz pola properties. Id wykorzystamy do *połączenia* naszych danych z mapą. Jeżeli chcemy znaleźć id konkretnego powiatu najlepiej wejść na stronę repozytorium.

### Przygotowanie danych
Musimy przygotować dane do połączenia z mapą. Dane przygotujemy w postaci jednej ramki danych z kolumną **id**, która reprezentuje powiat.

In [0]:
import pandas as pd
import numpy as np
import json
import requests

In [0]:
def read_patryk_files(url):
  raw_json = requests.request("GET", url).content
  data = json.loads(raw_json)['data']
  df =  pd.DataFrame(data, columns=['Data', 'Kwarantanna', 'Hospitalizowani', 'Potwierdzone przypadki', 'Ozdrowieńcy', 'Zgony'])
  df.loc[:, 'Data'] = pd.to_datetime(df['Data'],format='%d.%m.%Y')
  return df


url = "https://raw.githubusercontent.com/WilyLynx/Piekna_corona_python/master/data/minsk-psse.json"
minsk_df = read_patryk_files(url)
minsk_df['Powiat'] = ['miński' for i in range(len(minsk_df))] # tutaj dodaje nazwe powiatu
minsk_df['id'] = [217 for i in range(len(minsk_df))] # a tutaj id powiatu
minsk_df.head()

In [0]:
# To samo jeszcze dla dwóch powiatów
url = "https://raw.githubusercontent.com/WilyLynx/Piekna_corona_python/master/data/siedlce-psse.json"
siedlce_df = read_patryk_files(url)
siedlce_df['Powiat'] = ['siedlecki' for i in range(len(siedlce_df))]
siedlce_df['id'] = [120 for i in range(len(siedlce_df))]
siedlce_df.head()

In [0]:
url = "https://raw.githubusercontent.com/WilyLynx/Piekna_corona_python/master/data/lipsko-psse.json"
lipsko_df = read_patryk_files(url)
lipsko_df['Powiat'] = ['lipski' for i in range(len(lipsko_df))]
lipsko_df['id'] = [122 for i in range(len(lipsko_df))]
lipsko_df.head()

In [0]:
# Łączenie danych do jednej ramki
covid_df = pd.concat([minsk_df, siedlce_df, lipsko_df], axis=0)
covid_df['Ozdrowieńcy'].fillna(value=0, inplace=True)
covid_df['Zgony'].fillna(value=0, inplace=True)
covid_df['Aktywne'] = covid_df['Potwierdzone przypadki'] - covid_df['Ozdrowieńcy'] - covid_df['Zgony']
covid_df.head()

## Mapa 

Update plotly do najnowszej wersji okazał się niezbędny

In [0]:
pip install -U plotly

In [0]:
import plotly.express as px
single_day_df = covid_df[covid_df['Data'] == '2020-05-10']

fig = px.choropleth_mapbox(single_day_df, geojson=powiaty, locations='id', color='Aktywne',
                           featureidkey="properties.id",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=6.5, center = {"lat": 52.232222, "lon": 21.008333},
                           opacity=0.5,
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

Wykorzytajmy teraz właściwość animation_frame 

In [0]:
import plotly.express as px

covid_df.sort_values(by='Data', inplace=True, ascending=True) # brzydki sposób na to, żeby daty wyświetlały się w dobrej kolejności w animacji
covid_df['Data'] = [str(date) for date in covid_df['Data']]

fig = px.choropleth_mapbox(covid_df, geojson=powiaty, locations='id', color='Aktywne',
                           animation_frame  = 'Data',
                           range_color = (0, covid_df['Aktywne'].max()),
                           featureidkey="properties.id",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=6, center = {"lat": 52.232222, "lon": 21.008333},
                           opacity=0.5,
                           labels={'Aktywne':'Przypadki aktywne'}
                          )
fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    annotations = [dict(
        x=0.95,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Dane pozyskane ze stron powiatowych stacji epidemiologicznych',
        showarrow = False
    )]
    )


fig.show()